# Project 5 

Anne Louise Seekford  
Advanced Applied Machine Learning  
04.03.2022

## Comparison of Different Regularization and Variable Selection Techniques 
### In this project, you will apply and compare the different regularization techniques including Ridge, LASSO, Elastic Net, SCAD, and Square Root Lasso.

In [1]:
# Imports

import numpy as np
import pandas as pd
from math import ceil
from scipy import linalg
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.metrics import mean_absolute_error
from sklearn.datasets import make_spd_matrix
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from scipy.linalg import toeplitz
from matplotlib import pyplot
from numba import njit
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.model_selection import GridSearchCV, KFold

## 1) Create your own sklearn compliant functions for Square Root Lasso and SCAD so you could use them in conjunction with GridSearchCV for finding optimal hyper-parameters when data such as xx and yy are given.

### SCAD

In [2]:
# Define functions

@njit
def scad_penalty(beta_hat, lambda_val, a_val):
    is_linear = (np.abs(beta_hat) <= lambda_val)
    is_quadratic = np.logical_and(lambda_val < np.abs(beta_hat), np.abs(beta_hat) <= a_val * lambda_val)
    is_constant = (a_val * lambda_val) < np.abs(beta_hat)
    
    linear_part = lambda_val * np.abs(beta_hat) * is_linear
    quadratic_part = (2 * a_val * lambda_val * np.abs(beta_hat) - beta_hat**2 - lambda_val**2) / (2 * (a_val - 1)) * is_quadratic
    constant_part = (lambda_val**2 * (a_val + 1)) / 2 * is_constant
    return linear_part + quadratic_part + constant_part

@njit    
def scad_derivative(beta_hat, lambda_val, a_val):
    return lambda_val * ((beta_hat <= lambda_val) + (a_val * lambda_val - beta_hat)*((a_val * lambda_val - beta_hat) > 0) / ((a_val - 1) * lambda_val) * (beta_hat > lambda_val))

In [3]:
@njit
def scad(beta):
  beta = beta.flatten()
  beta = beta.reshape(-1,1)
  n = len(y)
  return 1/n*np.sum((y-x.dot(beta))**2) + np.sum(scad_penalty(beta,lam,a))

@njit  
def dscad(beta):
  beta = beta.flatten()
  beta = beta.reshape(-1,1)
  n = len(y)
  output = -2/n*np.transpose(x).dot(y-x.dot(beta))+scad_derivative(beta,lam,a)
  return output.flatten()

In [4]:
# SKLearn Compliant SCAD Function

class SCAD(BaseEstimator, RegressorMixin):
    def __init__(self, a=2,lam=1):
        self.a, self.lam = a, lam
  
    def fit(self, x, y):
        a = self.a
        lam   = self.lam

        @njit
        def scad(beta):
          beta = beta.flatten()
          beta = beta.reshape(-1,1)
          n = len(y)
          return 1/n*np.sum((y-x.dot(beta))**2) + np.sum(scad_penalty(beta,lam,a))

        @njit  
        def dscad(beta):
          beta = beta.flatten()
          beta = beta.reshape(-1,1)
          n = len(y)
          output = -2/n*np.transpose(x).dot(y-x.dot(beta))+scad_derivative(beta,lam,a)
          return output.flatten()
        
        
        beta0 = np.zeros(p)
        output = minimize(scad, beta0, method='L-BFGS-B', jac=dscad,options={'gtol': 1e-8, 'maxiter': 50000,'maxls': 50,'disp': False})
        beta = output.x
        self.coef_ = beta
        
    def predict(self, x):
        return x.dot(self.coef_)

### Square Root Lasso

In [5]:
# SKLearn Compliant SQRTLasso Function

class SQRTLasso(BaseEstimator, RegressorMixin):
    def __init__(self, alpha=0.01):
        self.alpha = alpha
  
    def fit(self, x, y):
        alpha=self.alpha
        @njit
        def f_obj(x,y,beta,alpha):
          n =len(x)
          beta = beta.flatten()
          beta = beta.reshape(-1,1)
          output = np.sqrt(1/n*np.sum((y-x.dot(beta))**2)) + alpha*np.sum(np.abs(beta))
          return output
        @njit
        def f_grad(x,y,beta,alpha):
          n=x.shape[0]
          p=x.shape[1]
          beta = beta.flatten()
          beta = beta.reshape(-1,1)
          output = (-1/np.sqrt(n))*np.transpose(x).dot(y-x.dot(beta))/np.sqrt(np.sum((y-x.dot(beta))**2))+alpha*np.sign(beta)
          return output.flatten()
        
        def objective(beta):
          return(f_obj(x,y,beta,alpha))
        
        def gradient(beta):
          return(f_grad(x,y,beta,alpha))
        
        beta0 = np.ones((x.shape[1],1))
        output = minimize(objective, beta0, method='L-BFGS-B', jac=gradient,options={'gtol': 1e-8, 'maxiter': 50000,'maxls': 25,'disp': True})
        beta = output.x
        self.coef_ = beta
        
    def predict(self, x):
        return x.dot(self.coef_)

## 2) Simulate 100 data sets, each with 1200 features, 200 observations and a toeplitz correlation structure such that the correlation between features *i* and *j* is approximately $p^{|i-j|}$ with $p = 0.8$.

 For the dependent variable y consider the following functional relationship:


** see on website **

In [6]:
n = 200 #num observations
p = 1200 #num features

In [7]:
beta_star = np.concatenate(([1]*7, [0]*25, [0.25]*5, [0]*50, [0.7]*15, [0]*1098))

In [8]:
# What we want to detect is the position of the actual information or "signal"

pos = np.where(beta_star != 0)

In [9]:
pos # shows the INDICIES where the value is not equal to zero

# 27 important conditions

(array([  0,   1,   2,   3,   4,   5,   6,  32,  33,  34,  35,  36,  87,
         88,  89,  90,  91,  92,  93,  94,  95,  96,  97,  98,  99, 100,
        101]),)

In [10]:
len(pos[0])

27

In [11]:
# What we need: toeplitz([1, 0.8, 0.8**2, 0.8**3, 0.8**4, ... , 0.8**1119])

v = []
for i in range(p):
  v.append(0.8**i)

In [ ]:
v

In [12]:
# Create covariance matrix (to use later when generating x)

mu = [0]*p #repeat 0, 1200 times
r = toeplitz(v) #covariance
sigma = 3.5 # per project instructions

# Generate the random samples
np.random.seed(123)
x = np.random.multivariate_normal(mu, r, size=n) #this is where we generate fictious data
y = np.matmul(x,beta_star) + sigma*np.random.normal(0,1,size=n)

In [63]:
y.shape #should be (200,1)

(200,)

## 3) Apply the variable selection methods that we discussed in-class such as Ridge, Lasso, Elastic Net, SCAD and Square Root Lasso with GridSearchCV (for tuning the hyper-parameters) and record the final results , such as the overall (on average) quality of reconstructing the sparsity pattern and the coefficients of $\beta^*$

The final results should include the average number of true non-zero coefficients discovered by each method, the L2 distance to the ideal solution and the Root Mean Squared Error.

#### Lasso

In [18]:
model_lasso = Lasso(alpha=0.1, fit_intercept=False) #False because we should not have an intercept because the mean of x is 0 and mean of y should be very very close to zero (by design)

In [20]:
model_lasso.fit(x,y)

Lasso(alpha=0.1, fit_intercept=False)

In [27]:
for param in gs_lasso.get_params().keys():
    print(param)

cv
error_score
estimator__alpha
estimator__copy_X
estimator__fit_intercept
estimator__max_iter
estimator__normalize
estimator__positive
estimator__precompute
estimator__random_state
estimator__selection
estimator__tol
estimator__warm_start
estimator
n_jobs
param_grid
pre_dispatch
refit
return_train_score
scoring
verbose


In [42]:
params = [{'alpha':np.linspace(0.001,1,num=50)}]

In [43]:
# GridSearch

gs_lasso = GridSearchCV(estimator=model_lasso,cv=10,scoring='neg_mean_squared_error',param_grid=params)
gs_las_results = gs_lasso.fit(x,y)
print(gs_las_results.best_params_)
print('LASSO mean square error is: ', np.abs(gs_las_results.best_score_))

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.837e+00, tolerance: 1.891e+00
  coef_, l1_reg, l2_reg, X, y, max_iter, tol, rng, random, positive
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.583e+00, tolerance: 1.933e+00
  coef_, l1_reg, l2_reg, X, y, max_iter, tol, rng, random, positive
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisa

{'alpha': 0.001}
LASSO mean square error is:  15.23773830528663


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.948e+00, tolerance: 2.145e+00
  coef_, l1_reg, l2_reg, X, y, max_iter, tol, rng, random, positive


LASSO mean square error is:  15.23773830528663

In [44]:
beta_hat_lasso = model_lasso.coef_

In [67]:
beta_hat_lasso

array([ 0.87672602,  2.60465118,  0.16529373, ..., -0.        ,
       -0.        , -0.        ])

Check quality of solution:

In [45]:
pos_lasso = np.where(beta_hat_lasso != 0 )

In [69]:
pos_lasso #indicies where betahat is not zero

(array([   0,    1,    2,    3,    4,    5,    6,    7,   18,   29,   32,
          35,   39,   58,   63,   64,   86,   87,   88,   89,   90,   92,
          94,   95,   96,   97,   99,  100,  101,  108,  133,  138,  143,
         151,  156,  163,  164,  168,  180,  184,  191,  192,  200,  243,
         249,  250,  258,  265,  270,  274,  285,  291,  297,  300,  301,
         314,  356,  367,  374,  382,  391,  401,  413,  459,  469,  475,
         480,  481,  482,  498,  503,  504,  515,  518,  538,  572,  575,
         582,  586,  588,  592,  630,  634,  646,  652,  660,  666,  672,
         684,  695,  717,  723,  724,  733,  734,  745,  758,  769,  770,
         779,  785,  805,  807,  824,  858,  860,  871,  876,  881,  897,
         904,  912,  920,  934,  947,  955,  973,  984,  993, 1003, 1010,
        1012, 1018, 1025, 1026, 1031, 1033, 1037, 1054, 1068, 1093, 1094,
        1134, 1140, 1141, 1142, 1146, 1147, 1159, 1166, 1167, 1187, 1194,
        1196]),)

In [46]:
np.array(pos_lasso).shape[1] #how many non zeros are there from this method

144

In [47]:
# Compare with "ground truth"

np.array(pos).shape[1]

27

In [48]:
# How many coefficients from Lasso belong to the real (pos) coffecients?

np.intersect1d(pos, pos_lasso)

array([  0,   1,   2,   3,   4,   5,   6,  32,  35,  87,  88,  89,  90,
        92,  94,  95,  96,  97,  99, 100, 101])

In [49]:
np.intersect1d(pos, pos_lasso).shape[0]

21

In [73]:
np.linalg.norm((bethatlasso-betastar), ord=2) #L2

In [ ]:
# MSE

# GRID SEARCH IS ALREADY VALIDATED

#### Ridge

In [39]:
model_ridge = Ridge(alpha=0.1, fit_intercept=False)

In [40]:
model_ridge.fit(x,y)

Ridge(alpha=0.1, fit_intercept=False)

In [50]:
gs_ridge = GridSearchCV(estimator=model_ridge,cv=10,scoring='neg_mean_squared_error',param_grid=params)
gs_ridge_results = gs_ridge.fit(x,y)
print(gs_ridge_results.best_params_)
print('RIDGE mean square error is: ', np.abs(gs_ridge_results.best_score_))

{'alpha': 0.001}
RIDGE mean square error is:  34.84857904255565


In [51]:
beta_hat_ridge = model_ridge.coef_

In [52]:
beta_hat_ridge

array([ 0.73288445,  0.81692647,  0.69790345, ..., -0.07933362,
       -0.05808182, -0.04977748])

Check quality of solution:

In [53]:
pos_ridge = np.where(beta_hat_ridge != 0 )

In [79]:
pos_ridge #indicies where betahat is not zero

(array([   0,    1,    2, ..., 1197, 1198, 1199]),)

In [54]:
np.array(pos_ridge).shape[1] #how many non zeros are there from this method

1200

In [55]:
# Compare with "ground truth"

np.array(pos).shape[1]

27

In [56]:
# How many coefficients from Lasso belong to the real (pos) coffecients?

np.intersect1d(pos, pos_ridge)

array([  0,   1,   2,   3,   4,   5,   6,  32,  33,  34,  35,  36,  87,
        88,  89,  90,  91,  92,  93,  94,  95,  96,  97,  98,  99, 100,
       101])

In [57]:
np.intersect1d(pos, pos_ridge).shape[0]

27

#### Elastic Net

In [28]:
model_en = ElasticNet(alpha=0.1, fit_intercept=False)

In [29]:
model_en.fit(x,y)

ElasticNet(alpha=0.1, fit_intercept=False)

In [30]:
gs_en = GridSearchCV(estimator=model_en,cv=10,scoring='neg_mean_squared_error',param_grid=params)
gs_en_results = gs_en.fit(x,y)
print(gs_en_results.best_params_)
print('Elastic Net mean square error is: ', np.abs(gs_en_results.best_score_))

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.752e+00, tolerance: 1.891e+00 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  coef_, l1_reg, l2_reg, X, y, max_iter, tol, rng, random, positive
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.818e+00, tolerance: 1.933e+00 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented

{'alpha': 0.001, 'l1_ratio': 0.7551020408163265}
LASSO mean square error is:  14.823625894539939


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.910e+00, tolerance: 2.145e+00
  coef_, l1_reg, l2_reg, X, y, max_iter, tol, rng, random, positive


Elasic Net mean square error is:  14.823625894539939

In [31]:
beta_hat_en = model_en.coef_

In [32]:
beta_hat_en

array([ 1.12643364,  1.53685203,  0.72808469, ..., -0.        ,
       -0.        , -0.        ])

Check quality of solution:

In [33]:
pos_en = np.where(beta_hat_en != 0 )

In [ ]:
pos_en #indicies where betahat is not zero

In [35]:
np.array(pos_en).shape[1] #how many non zeros are there from this method

238

In [36]:
# Compare with "ground truth"

np.array(pos).shape[1]

27

In [37]:
# How many coefficients from Lasso belong to the real (pos) coffecients?

np.intersect1d(pos, pos_en)

array([  0,   1,   2,   3,   4,   5,   6,  32,  34,  35,  87,  88,  89,
        90,  91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101])

In [38]:
np.intersect1d(pos, pos_en).shape[0]

25

#### SCAD

In [15]:
p = x.shape[1]
b0 = np.zeros(p)

In [ ]:
lam = 1
a = 2
output = minimize(scad, b0, method='L-BFGS-B', jac=dscad,options={'gtol': 1e-8, 'maxiter': 50000,'maxls': 50,'disp': True})

In [19]:
yhat_test_scad = x.dot(output.x)

NameError: ignored

In [17]:
model_sc = SCAD(a=2,lam=1)

In [18]:
model_sc.fit(x,y)

ValueError: ignored

In [61]:
params = [{'alpha':np.linspace(0.001,1,num=50),'l1_ratio':np.linspace(0,1,num=50)}]

In [ ]:
# GridSearch

gs_sc = GridSearchCV(estimator=model_sc,cv=10,scoring='neg_mean_squared_error',param_grid=params)
gs_sc_results = gs_sc.fit(x,y)
print(gs_sc_results.best_params_)
print('SCAD mean square error is: ', np.abs(gs_sc_results.best_score_))

In [ ]:
beta_hat_sc = model_sc.coef_

In [ ]:
beta_hat_sc

array([ 1.12643364,  1.53685203,  0.72808469, ..., -0.        ,
       -0.        , -0.        ])

Check quality of solution:

In [ ]:
pos_sc = np.where(beta_hat_sc != 0 )

In [ ]:
pos_sc #indicies where betahat is not zero

(array([   0,    1,    2, ..., 1197, 1198, 1199]),)

In [ ]:
np.array(pos_sc).shape[1] #how many non zeros are there from this method

1200

In [ ]:
# Compare with "ground truth"

np.array(pos).shape[1]

27

In [ ]:
# How many coefficients from SCAD belong to the real (pos) coffecients?

np.intersect1d(pos, pos_sc)

array([  0,   1,   2,   3,   4,   5,   6,  32,  33,  34,  35,  36,  87,
        88,  89,  90,  91,  92,  93,  94,  95,  96,  97,  98,  99, 100,
       101])

In [ ]:
np.intersect1d(pos, pos_sc).shape[0]

#### Square Root Lasso w/Grid Search CV

In [70]:
params = [{'alpha':np.linspace(0.001,1,num=50)}]

In [67]:
model_sqrt = SQRTLasso(alpha=0.1)

In [ ]:
model_sqrt.fit(x,y)

In [ ]:
gs_sqrt = GridSearchCV(estimator=model_sqrt,cv=10,scoring='neg_mean_squared_error',param_grid=params)
gs_sqrt_results = gs_sqrt.fit(x,y)
print(gs_sqrt_results.best_params_)
print('Square Root Lasso mean square error is: ', np.abs(gs_sqrt_results.best_score_))

Elasic Net mean square error is:  14.823625894539939

In [ ]:
beta_hat_en = model_en.coef_

In [ ]:
beta_hat_en

array([ 1.12643364,  1.53685203,  0.72808469, ..., -0.        ,
       -0.        , -0.        ])

Check quality of solution:

In [ ]:
pos_en = np.where(beta_hat_en != 0 )

In [ ]:
pos_en #indicies where betahat is not zero

In [ ]:
np.array(pos_en).shape[1] #how many non zeros are there from this method

238

In [ ]:
# Compare with "ground truth"

np.array(pos).shape[1]

27

In [ ]:
# How many coefficients from Lasso belong to the real (pos) coffecients?

np.intersect1d(pos, pos_en)

array([  0,   1,   2,   3,   4,   5,   6,  32,  34,  35,  87,  88,  89,
        90,  91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101])

In [ ]:
np.intersect1d(pos, pos_en).shape[0]

25